# Download data

In [ ]:
import json
#Download lyrics
!gdown --id 1V7WxGaVv3UamD4AM5qnqDNzkBSw6paQy
#Download queries
!gdown --id 1srNAjKMTncZUgxDTZZHfMnTbPd9ic2xx

# Preliminaries & Corpus Creation

In [ ]:
# Install required libraries
!pip install rank-bm25

In [ ]:
!pip install -q tf-models-official==2.3.0

     |████████████████████████████████| 840 kB 4.2 MB/s 
     |████████████████████████████████| 99 kB 8.7 MB/s 
     |████████████████████████████████| 1.1 MB 43.8 MB/s 
     |████████████████████████████████| 352 kB 46.9 MB/s 
     |████████████████████████████████| 1.2 MB 33.2 MB/s 
     |████████████████████████████████| 234 kB 47.3 MB/s 
     |████████████████████████████████| 47.8 MB 83 kB/s 
     |████████████████████████████████| 462 kB 14.2 MB/s 


In [ ]:
import pandas as pd


In [ ]:

df = pd.read_json (r'/content/2335_Songs.json')
df.to_csv (r'/content/sample_data/data.csv', index = None)

In [ ]:
df["song_id"].head(100)

In [ ]:
# Import required libraries
import os
import pprint as pp
import numpy as np
import json
import tensorflow as tf 
from gensim.corpora import WikiCorpus
from rank_bm25 import BM25Okapi, BM25Plus
from official.modeling import tf_utils
from official import nlp
from official.nlp import bert

# Load the required submodules
import official.nlp.optimization
import official.nlp.bert.bert_models
import official.nlp.bert.configs
import official.nlp.bert.run_classifier
import official.nlp.bert.tokenization
import official.nlp.data.classifier_data_lib
import official.nlp.modeling.losses
import official.nlp.modeling.models
import official.nlp.modeling.networks

In [ ]:
from tqdm import tqdm
# link to data
CORPUS_SONG_JSON = "2335_Songs.json"
QUERIES_JSON = "500_queries.json"

In [ ]:
with open(CORPUS_SONG_JSON, 'r', encoding='utf-8') as input:
  corpus_data = json.load(input)

with open(QUERIES_JSON, 'r', encoding='utf-8') as input:
  queries_test = json.load(input)

In [ ]:
corpus_data[987]

In [ ]:
type(corpus_data)

In [ ]:
import re
import sys

patterns = {
    '[àáảãạăắằẵặẳâầấậẫẩ]': 'a',
    '[đ]': 'd',
    '[èéẻẽẹêềếểễệ]': 'e',
    '[ìíỉĩị]': 'i',
    '[òóỏõọôồốổỗộơờớởỡợ]': 'o',
    '[ùúủũụưừứửữự]': 'u',
    '[ỳýỷỹỵ]': 'y'
}

def convert(text):
    output = text
    for regex, replace in patterns.items():
        output = re.sub(regex, replace, output)
        # deal with upper case
        output = re.sub(regex.upper(), replace.upper(), output)
    return output

In [ ]:
strash_phrases = ['intro', 'verse', 'ver', 'hook',"[db]",'[ab]', 'd/k', 'verse2',  'bridge', 'ohwell', 'melody', 'ver1', 'jokesbii', 'ver2', 'wegetlow', 'inthesky', 'hook', 'brigde', 'vers2', 'verse1', 'x2', 'verwolf','coming', 'drop', 'đk','jser', 'dk', 'bitch', 'imhere', 'chorus', 'midtrox2', 'ver2x2', 'hook1', 'igotmove', '|đk|', 'slick', 'x3', 'x4', 'hpro', 'supb', 'prehook', '1d', 'x8', 'slowx2', '2ndver', 'v1', 'v2','vers1', 'mel1', 'mel2', 'trpl5', 'roostr', 'hookx2','high',  'đk1', 'đk2', 'pre', 'rap', 'tia', 'ver3', 'ver4', 'ver5', 'ver6', 'ver7', 'vers', 'verse3', 'choem', 'brige', 'atmyshow', 'melody1', 'melody2', 'ohmysign',  'melodic', 'jd','midtro', 'ver2jd','dropdown', 'hook2', 'hook3', 'bmtk', 'vsplifff',  'bemygirl', 'dk2', 'neon',  'verkyle', 'vers3', 'kellyver', 'keepgoon', 'dng', 'outro', 'precho', 'misery', 'sohigh', 'mid', 'deep', 'getlittt', 'lietome', 'lvking','tri', 'astrole', 'mkid', 'bigsmile', 'roki','bridge1', 'xinem', 'verrse2', 'tome', 'robochop', 'verrap1', 'verrap2', 'ntro', 'hold', 'chords','drwild', 'royp', 'chunn', 'pre–hook', 'diorlogo', 'allthem', 'dmyb', 'verbitu', 'likethis', 'pre1', 'cho1', 'pre2', 'đk3', 'kingperu', 'hidan', 'biosap', 'bentley', 'priceless', 'goingwild', 'lilbtch', 'emflexing', 'kdv',  'storkh', 'freshlyrc', 'ohmygirl',  'mcape', 'y6mz', 'befriend', 'rever', 'slowdown', 'toxic', 'loser'] #, 'chorus', 'pre-chorus']
source = [" t "," ik ", ' mik ', ' nayd ',' ko ',"in'",'dzề']
change = [" tao "," đi ", ' mình ', ' này ',' không ','ing','về']

In [ ]:
for i in range(2335):
  corpus_data[i]['author'] = corpus_data[i]['author'].lower()
  corpus_data[i]['title'] = corpus_data[i]['title'].lower()
  corpus_data[i]['lyrics'] = corpus_data[i]['lyrics'].lower()

  for phrase in strash_phrases:
   corpus_data[i]['lyrics']=corpus_data[i]['lyrics'].replace(phrase,"")
  for j in range(len(source)):
    corpus_data[i]['lyrics']=corpus_data[i]['lyrics'].replace(source[j],change[j])
  corpus_data[i]['lyrics'] = convert(corpus_data[i]['lyrics'])


In [ ]:
corpus_data

[{'author': 'trần bình',
  'lyrics': 'bai hat: tinh yeu thuy chung - tran binh, viet linh\nco phai la duyen khi anh gap em\ntu kiep truoc, the uoc chung doi\ncung han hoan\nnhin nhau dam duoi\nlong quyen luyen,\nchuyen mai khong thoi\nem tu noi dau da den noi nay\nanh han hoan\ntrao em vong tay\nva doi ta,\nsong chung hanh phuc\ncung xay dap,\ncho moi duyen nay\nem oi em oi, du phong ba\ndu bao to, du nang chay\nanh van yeu em, mai mai khong thoi\nanh oi anh oi, du gian nan\ndu khon kho ta van yeu nhau,\nden cuoi cuoc doi\nta van ben nhau qua bao buon vui\nta yeu nhau, tinh yeu dam say\ntinh yeu ta dep, tinh yeu tham thiet\nhanh phuc tram nam\ntinh yeu thuy chung\n \n',
  'song_id': '0000',
  'title': 'tình yêu thuỷ chung'},
 {'author': 'trần bình',
  'lyrics': 'bai hat: la thu cua cha - tran binh\ntu ngay con ra di\ncha long buon thuong nho\nco don va sau muon\nbao ngay thang mong cho\ntu ngay con ra di\ntoi thoi gian bay gio\ntham thoat muoi hai nam chan\ncha thuong dung ngoi ngan ng

In [ ]:
def read_corpus(corpus_data):
    """
    Function that iteratively reads the saved articles from the corpus directory
    and appends the text to a list.
    
    Parameters
    ----------
    @param corpus_directory: str,
        A valid string specifying the path to the local directory in which the 
        files were saved using make_corpus().
        
    Returns
    -------
    @return corpus, list
        A list containing the text of an article in each element.
    """
    
    # Initialize empty corpus list
    corpus = []
    
    # For each file in the corpus directory, do...
    print("Reading local corpus, please wait...")

    for song in corpus_data:
      if song["lyrics"].split("\n")[0].startswith("Bài hát: "):
          lyric = "\n".join(song["lyrics"].split("\n")[1:])
      else:
          lyric = song["lyrics"]
      corpus.append([song["song_id"], lyric])
        
    # Done, return
    print("Done!")
    return corpus

In [ ]:
# Read corpus! 
corpus = read_corpus(corpus_data)

# Look at some example...
corpus[2]

# For some reason (only in Google Colab), this cell might need to be stopped
# for the first run, then run again. 

# BM25 Implementation

In [ ]:
!pip install underthesea
!pip install pyvi

In [ ]:
# Tokenize the corpus
from underthesea import word_tokenize

tokenized_corpus = []
for doc in tqdm(corpus):
  tokenize_lyric = [t for t in doc[1].split('\n')]
  tokenize_lyric.append(f"-->{doc[0]}")
  tokenized_corpus.append(tokenize_lyric)

In [ ]:
tokenized_corpus

In [ ]:
# Instantiate BM25 object from the tokenized corpus
bm25 = BM25Okapi(tokenized_corpus)

In [ ]:
#hàm a nhận một truy vấn chuỗi và đánh số `n` kết quả bắt buộc, 
#và trả về kết quả có liên quan nhất` n` từ kho dữ liệu của chúng ta:
def bm25okapi_search(tokenized_query, bm25, corpus, n_results = 1):
    """
    Function that takes a tokenized query and prints the first 100 words of the 
    n_results most relevant results found in the corpus, based on the BM25
    method.
    
    Parameters
    ----------
    @param tokenized_query: list, array-like
        A valid list containing the tokenized query.
    @param bm25: BM25 object,
        A valid object of type BM25 (BM25Okapi or BM25Plus) from the library
        `rank-bm25`, initialized with a valid corpus.
    @param corpus: list, array-like
        A valid list containing the corpus from which the BM25 object has been 
        initialized. As returned from function read_corpus().
    @param n_results: int, default = 1
        The number of top results to print.
    """
    
    # We skip checking validity of arguments for now... We assume the user 
    # knows what they're doing.
    
    # Get top results for the query
    top_results = bm25.get_top_n(tokenized_query, corpus, n = n_results)
    
    # Take words from each result
    top_results_id = [' '.join(top_result).split("-->")[-1] 
                             for top_result in top_results]
    
    # Return results
    return top_results_id

In [ ]:
# check queries
for i in range(500):
  queries_test[i]['query'] = queries_test[i]['query'].lower()

  for phrase in strash_phrases:
   queries_test[i]['query']=queries_test[i]['query'].replace(phrase,"")
  for j in range(len(source)):
    queries_test[i]['query']=queries_test[i]['query'].replace(source[j],change[j])
  queries_test[i]['query']=queries_test[i]['query'].replace("  "," ")
  queries_test[i]['query']= convert(queries_test[i]['query'])

In [ ]:
queries_test

[{'query': 'cho qua bao năm mộng buon quen dấu', 'query_id': '000'},
 {'query': 'chuyện mình giờ kết thúc thật rồi ', 'query_id': '001'},
 {'query': 'dưới hiên nhà nhìn mưa rơi', 'query_id': '002'},
 {'query': 'đi ngang qua oh my lady', 'query_id': '003'},
 {'query': 'gác lại mọi thu / cua ngày hôm qua .', 'query_id': '004'},
 {'query': 'bóng khói lam chiều dập diều mây bây nhấp nhô.',
  'query_id': '005'},
 {'query': "(yeah yeah, chúng m nên nhớ một điều là vậy, man, it's true)",
  'query_id': '006'},
 {'query': 'tối nghe nhung bai lofi ma em so like va nghe chuyen em ke',
  'query_id': '007'},
 {'query': 'tinh cam nay đã hết thật roi ', 'query_id': '008'},
 {'query': 'gom gop yeu thuong que nha,dang het cho người tình xa !',
  'query_id': '009'},
 {'query': 'vì trốn ngủ trưa ở ngoài đuong, ao ba lỗ và ra nắng',
  'query_id': '010'},
 {'query': 'ngon ngon hôm nay có món ngon ngon', 'query_id': '011'},
 {'query': 'du cho em co sai mai thoi cu làm lại.', 'query_id': '012'},
 {'query': '

In [ ]:
corpus[383][1]

'sài gòn dù đã như ngày xưa mà lòng quặn thắt sao lạ ghê người người rời chốn hoa lệ trở về quê khóe mắt tôi mặn cay đau nhói nhìn đàn bé con nay, k còn cha rồi mẹ giờ cũng theo cha, đi thật xa để lại ngàn xót xa em thơ, bơ vơ có nỗi đau nào hơn, kiếp mồ côi sài gòn giờ đây bão giông đã đi qua sài gòn giờ đây lệ đắng k phai nhòa một nhành hoa trắng mãi mãi luôn trên ngực còn gì đau hơn sự mất mát sài gòn rồi sẽ hơn ngày xưa người người rồi sẽ quay lại đây chào một ngày mới với muôn ngàn điều hay sẽ tái sinh sài gòn tôi rạng ngời  '

# Process & Export file submission

# New Section

In [ ]:
import csv
from pyvi import ViUtils
import re
lst = []
with open("submission1.csv", "w") as csv_writer:
  fieldnames = ['query_id', 'song_id']
  writer = csv.DictWriter(csv_writer, fieldnames=fieldnames)
  writer.writeheader()
  for ques in tqdm(queries_test):
    preprocess_query = " ".join(ques["query"].lower().strip("").split())
    tokenized_query = [t for t in preprocess_query.split("\n")]
    results = bm25okapi_search(tokenized_query = tokenized_query, bm25 = bm25, corpus = tokenized_corpus, n_results = 10)
    writer.writerow({'query_id': ques["query_id"], "song_id": " ".join(results)})

100%|██████████| 500/500 [00:01<00:00, 339.60it/s]


In [ ]:
s = 'chuYỆN MÌnH Giờ kết thúc thật rồi '
preprocess_query = " ".join(s.lower().strip("").split())
preprocess_query

'chuyện mình giờ kết thúc thật rồi'